# RNN (循环神经网络)

以前我们写过的 Network 的结果都是反馈给下一层。

那在RNN中，我们不仅要输出结果，还要输出一个**隐含状态**，给当前层在处理下一个样本时使用。

结构图如下所示：

![Recurrent Neural Networks have loops](../images/RNN/RNN-rolled.jpg)


可以看到，跟普通的 Neural Network 相比，我们这里仅仅是多了一个**隐含状态**，那么这个多出来的**隐含状态**有什么好处呢？

这个**隐含状态**，我们可以看作是上一个样本遗留给我们的信息，通过这个信息，我们可以把握过去样本的特征，结合当前的样本，组成一种类似于 “时间序列” 样本信息，从而让模型有了处理前后有依赖关系数据样本的能力。

**举个例子:**

语言模型的任务是给定句子的前 $t$ 个字符，然后预测第 $t+1$ 个字符。假设我们的句子是“你好世界”，使用循环神经网络来预测的一个做法是，在时间 $1$ 输入“你”，预测“好”同时生成一个**隐含状态**；

在时间 $2$ 向同一个网络输入刚才生成的**隐含状态** 和 “好” 去预测“世”。同时输出一个更新过的隐含状态。

对于这个**隐含状态**， 我们会希望前面的信息能够保存在这个隐含状态里，从而提升预测效果。下图展示了这个过程。

![Recurrent Neural Networks have loops](../images/RNN/RNN.jpg)




# RNN的公式表示

针对输入输出加上时间状态 $t$。在 $t$ 时刻，我们的输入为 $X_t$ 和 $H_{t-1}$，经过一个隐含层之后，我们有输出 $H_t$，经过最后的输出层得到 $\hat{Y_t}$。

其中 $X_t \in R^{n \times x}$ (样本数量为 $n$，每个样本特征向量维度是 $x$) ； $H_{t-1} \in R^{n \times h}$ （其中隐含层长度为 $h$）

隐含层的计算公式为：

$$\mathbf{H}_t = \phi(\mathbf{X}_t \mathbf{W}_{xh} + \mathbf{H}_{t-1} \mathbf{W}_{hh}  + \mathbf{b}_h)$$

其中 $\mathbf{W}_{xh}$ 和 $\mathbf{W}_{hh}$ 是隐含层的权重， $\mathbf{b}_h$ 是隐含层的 bias。

输出层的计算公式为：

$$\hat{\mathbf{Y}}_t = \text{softmax}(\mathbf{H}_t \mathbf{W}_{hy}  + \mathbf{b}_y)$$

其中 $\mathbf{W}_{hy}$ 是输出层权重，$\mathbf{b}_y$ 是输出层的 bias


最开始的隐含状态里的元素通常会被初始化为0，也就是 $H_0$ 是个零矩阵。






循环神经网络的简单介绍就到这里了，如果想要更加细致的了解，可以看本人写过的一篇Blog [详细剖析RNN理论、变种和应用](https://lianhaimiao.github.io/2018/02/09/%E8%AF%A6%E7%BB%86%E5%89%96%E6%9E%90RNN%E7%90%86%E8%AE%BA%E3%80%81%E5%8F%98%E7%A7%8D%E5%92%8C%E5%BA%94%E7%94%A8/#more)

# RNN的 PyTorch 实现

理论部分我们已经了解了，那么下实际操作中，我们如何用 PyTorch 写一个RNN呢？


答案很简单，就是一个函数...

```
torch.nn.RNN()
```

对的，你没看错，实现 RNN 就是这么简单。

在 pyTroch 中你实现 RNN 模型的时候，可以传入的参数有：

> input_size – The number of expected features in the input x

> hidden_size – The number of features in the hidden state h

> num_layers – Number of recurrent layers.

> nonlinearity – The non-linearity to use [‘tanh’|’relu’]. Default: ‘tanh’

> bias – If False, then the layer does not use bias weights b_ih and b_hh. Default: True

> batch_first – If True, then the input and output tensors are provided as (batch, seq, feature)

> dropout – If non-zero, introduces a dropout layer on the outputs of each RNN layer except the last layer

> bidirectional – If True, becomes a bidirectional RNN. Default: False


在这里我们需要重点关注的是 **input_size**、**hidden_size**、**batch_first**

这三个分别表示，输入数据的特征大小、隐含层特征大小、batch放在第一位。

模型实现之后，就是输入数据的格式了。如果在模型实现的时候选择了**batch_first=True**，那么在输入数据的格式就会是 (batch, seq_len, input_size)


OK，整个RNN模型的 PyTorch 的理论部分，先介绍到这里，接下来的练习是**使用 RNN 模型来生成恐龙名字**


# 使用 RNN 模型来生成恐龙名字

**PS：该练习的原始数据和想法来自于 吴恩达 Deep Learning 系列课程第五课**

## 第一步：字符的数值表示 和 One-Hot Encoding

因为字符没办法输入到模型里面，所以我们得先把数据里面所有不同的字符拿出来做成一个字典，然后可以把每个字符转成从0开始的索引(index)来方便之后的使用。

需要注意的是我们一共有 27 个字符，出了小写字母 a-z(26个) 之外，我们还会有一个字符就是 "\n" 这个字符在这里的含义是 < EOS \> (or "End of sentence") 







In [3]:
data = open('../dataSet/dinos.txt', 'r').read()
data= data.lower()
chars = list(set(data))
data_size, vocab_size = len(data), len(chars)
print('数据集中一共有 %d 个字符，不同的字符一共有 %d 个。' % (data_size, vocab_size))

数据集中一共有 19909 个字符，不同的字符一共有 27 个。


In [4]:
char_to_ix = { ch:i for i,ch in enumerate(sorted(chars)) } # 字符到索引的映射
ix_to_char = { i:ch for i,ch in enumerate(sorted(chars)) } # 索引到字符的映射
print(ix_to_char)

{0: '\n', 1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z'}


现在，我们已经得到了 字符转数字 和 数字转字符的映射了，但是，这个现在的每个字符依旧是用一个整数表示的，而输入进网络我们需要一个定长的向量。一个常用的办法是使用**One-Hot Encoding**来将其表示成向量。也就是说，如果一个字符的整数值是 $i$ , 那么我们创建一个全0的长为 vocab_size 的向量，并将其第 $i$ 位设成1。该向量就是对原字符的one-hot向量。

## 第二步：梯度裁剪

## 第三步：随机采样

## 第四步：构建模型